<a href="https://colab.research.google.com/github/sigopt/sigopt-examples/blob/main/metric-constraints-demo/sigopt_metric_constraints_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab Notebook showcases the [Metric Constraints](https://app.sigopt.com/docs/overview/metric_constraints) feature in SigOpt, as described in [this blog post](https://sigopt.com/blog/metric-constraints-demo/). We use the Metric Constraints feature to optimize for the top-1 accuracy of a CNN with a constraint of the size of the network. We demonstrate this feature using the German Traffic Signs Dataset (GTSRB).

In [ ]:
! pip install sigopt

In [ ]:
from copy import deepcopy
import numpy
import pickle
from sigopt import Connection
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

Loading Data. Refer to [this colab notebook](https://colab.research.google.com/drive/17wpJYYMOIzbnaW18Wn7dXT91ahRIJ0N_) on the data augmentation pipeline.

In [ ]:
relative_path = "./drive/My Drive/Colab Notebooks/"
training_file = relative_path + "traffic-signs-data/train_extended.p"
validation_file= relative_path + "traffic-signs-data/valid.p"
testing_file = relative_path + "traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
  train = pickle.load(f)
with open(validation_file, mode='rb') as f:
  valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
  test = pickle.load(f)

In [ ]:
X_train = train['features']
y_train = train['labels']
X_valid = valid['features']
y_valid = valid['labels']
X_test = test['features']
y_test = test['labels']

In [ ]:
# Sometimes the validation set and testing set images are saved with intensity level of [0, 255]. Convert these to [0, 1].
if X_valid.dtype == numpy.uint8:
  X_valid = (X_valid / 256).astype('float32')
if X_test.dtype == numpy.uint8:
  X_test = (X_test / 256).astype('float32')

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# Data constant variables
NUM_CLASSES = 43
IMG_SIZE = 32
# Training constant variables
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
y_train_cat = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_valid_cat = keras.utils.to_categorical(y_valid, NUM_CLASSES)
y_test_cat = keras.utils.to_categorical(y_test, NUM_CLASSES)

Setting up the Keras model, parameterizing the hyperparameters that we want to tune, and evaluating the metrics that we want to track, optimize, or constrain. The CNN model is inspired by the [*MicronNet*](https://arxiv.org/abs/1804.00497) model.

In [ ]:
class TrainingTimeLogger(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.time = 0
    self.start_time = time.time()

  def on_train_end(self, logs={}):
    self.time = time.time() - self.start_time

def define_munet(hps):
  fc_1 = int(hps['fc_1'])
  fc_2 = int(hps['fc_2'])
  kernel_size_1 = int(hps['kernel_size_1'])
  kernel_size_2 = int(hps['kernel_size_2'])
  kernel_size_3 = int(hps['kernel_size_3'])
  num_filters_1 = int(hps['num_filters_1']) 
  num_filters_2 = int(hps['num_filters_2'])
  num_filters_3 = int(hps['num_filters_3'])

  model = tf.keras.Sequential()
  model.add(layers.Conv2D(3, (1, 1), input_shape=(IMG_SIZE, IMG_SIZE, 3),activation='relu'))
  model.add(layers.Conv2D(num_filters_1, (kernel_size_1, kernel_size_1), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  model.add(layers.Conv2D(num_filters_2, (kernel_size_2, kernel_size_2), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  model.add(layers.Conv2D(num_filters_3, (kernel_size_3, kernel_size_3), padding='same', activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=2))

  model.add(layers.Flatten())
  model.add(layers.Dense(fc_1, activation='relu'))
  model.add(layers.Dense(fc_2, activation='relu'))
  model.add(layers.Dense(NUM_CLASSES, activation='softmax'))
  return model

def train_model(model):
  time_callback = TrainingTimeLogger()
  with tf.device('/device:GPU:0'):
    opt = tf.keras.optimizers.SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
    model.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['categorical_accuracy']
    )
    datagen = ImageDataGenerator()
    history = model.fit_generator(datagen.flow(
      X_train, y_train_cat, batch_size=32),
      steps_per_epoch=len(y_train) // 32,
      epochs=EPOCHS,
      validation_data=(X_valid, y_valid_cat),
      callbacks=[time_callback],
      shuffle=True,
      verbose=2,
    )
    train_time = time_callback.time
    validation_accuracy = history.history['val_categorical_accuracy'][-1]
    test_accuracy = model.evaluate(X_test, y_test_cat)[1]
  return validation_accuracy, test_accuracy, train_time, history


MIN_ACCEPTABLE_VAL_ACCURACY = 0.2

def create_observation(suggestion):
  model = None
  size = 0
  val_accuracy = 0
  test_accuracy = 0
  training_time = 0
  try:
    model = define_munet(suggestion.assignments)
    size = model.count_params() / 1e6
    val_accuracy, test_accuracy, training_time, history = train_model(model)
  except ValueError as e:
    print(f'ValueError {e} with {suggestion.assignments.values()}')
    return {
      'suggestion': suggestion.id,
      'failed': True,
      'metadata': dict(
          error_msg=e,
      )
    }
  # Sometimes the model diverges, going to mark these as failures instead
  if val_accuracy <= MIN_ACCEPTABLE_VAL_ACCURACY:
    return {
      'suggestion': suggestion.id,
      'failed': True,
      'metadata': dict(
        error_msg='divergence',
        validation_accuracy=val_accuracy,
        training_time=training_time,
        loss=repr(["%.5f" % l for l in history.history['loss']])
      )
    }
  return {
    'suggestion': suggestion.id,
    'values': [
      {'name': 'size', 'value': size},     
      {'name': 'validation_accuracy', 'value': val_accuracy},
      {'name': 'test_accuracy', 'value': test_accuracy},
      {'name': 'training_time', 'value': training_time},
    ],
  }

Setting up the SigOpt experiment.

In [ ]:
experiment_meta = dict(
  name="Traffic Dataset, Constraint Metric v2",
  parameters=[
    dict(name="kernel_size_1", bounds=dict(min=2, max=7), type="int"),
    dict(name="kernel_size_2", bounds=dict(min=2, max=7), type="int"),
    dict(name="kernel_size_3", bounds=dict(min=2, max=7), type="int"),
    dict(name="num_filters_1", bounds=dict(min=10, max=50), type="int"),
    dict(name="num_filters_2", bounds=dict(min=30, max=70), type="int"),
    dict(name="num_filters_3", bounds=dict(min=40, max=160), type="int"),
    dict(name="fc_1", bounds=dict(min=10, max=1000), type="int"),
    dict(name="fc_2", bounds=dict(min=10, max=1000), type="int"),
  ],
  metrics = [
    dict(
      name='size',
      objective='minimize',
      strategy='constraint',
      threshold=0.25
    ),
    dict(
      name='validation_accuracy',
      objective='maximize',
      strategy='optimize',
    ),
    dict(
      name='test_accuracy',
      objective='maximize',
      strategy='store',
    ),
     dict(
      name='training_time',
      objective='minimize',
      strategy='store',
    ),
  ],
  metadata=dict(
    training_file=training_file,
    validation_file=validation_file,
    testing_file=testing_file,
    environment='Tesla P100-PCIE',
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    min_acceptable_accuracy=MIN_ACCEPTABLE_VAL_ACCURACY
  ),
  observation_budget=200,
  parallel_bandwidth=1,
  project='cm-blog'
)

In [ ]:
conn = Connection(client_token='YOUR_SIGOPT_API_TOKEN')

In [ ]:
experiment = conn.experiments().create(**experiment_meta)
# experiment.id
print(f'Created experiment: https://app.sigopt.com/experiment/{experiment.id}')

In [ ]:
for i in range(experiment.observation_budget):
  s = conn.experiments(experiment.id).suggestions().create()
  obs = create_observation(s)
  conn.experiments(experiment.id).observations().create(**obs)

Updating the threshold

In [ ]:
experiment = conn.experiments(experiment.id).update(
  metrics = [
    dict(
      name='size',
      threshold=0.15
    ),
    dict(
      name='validation_accuracy',
    ),
    dict(
      name='test_accuracy',
    ),
     dict(
      name='training_time',
    ),
  ],
)